In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# Import dependencies
from tensorflow.keras.preprocessing import image_dataset_from_directory
import tensorflow as tf

#data.py
# Insert your local path to train data and test data here:
path_to_train_data = '/content/drive/MyDrive/LeWagon/AlzheimersDataset/train'
path_to_test_data =  '/content/drive/MyDrive/LeWagon/AlzheimersDataset/test'

def train_data_loading(path_to_train_data):

    # Train Generator
    train_generator = image_dataset_from_directory(
    path_to_train_data,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    labels = 'inferred',
    label_mode = 'categorical',
    shuffle = True,
    validation_split = 0.2,
    subset = 'training',
    seed = 123)

    # Validation Generator
    validation_generator = image_dataset_from_directory(
    path_to_train_data,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    labels = 'inferred',
    label_mode = 'categorical',
    shuffle = True,
    validation_split = 0.2,
    subset = 'validation',
    seed = 123)

    return train_generator, validation_generator

def test_data_loading(path_to_test_data):

    # Test Generator
    test_generator = image_dataset_from_directory(
    path_to_test_data,
    image_size = IMAGE_SIZE,
    batch_size = BATCH_SIZE,
    labels = 'inferred',
    label_mode = 'categorical',
    shuffle = True,
    seed = 123)

    return test_generator

#model.py
from tensorflow.keras import Sequential, layers
from tensorflow.keras.applications.densenet import DenseNet121

def build_compile_model():

    # DenseNet121 Base Model
    base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=INPUT_SHAPE)
    base_model.trainable = False

    # Base Model + Trainable Layers
    model = Sequential()
    model.add(base_model)
    model.add(layers.Flatten())
    model.add(layers.Dense(50, activation='relu'))
    model.add(layers.Dense(4, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS)

    return model

#params.py
import tensorflow as tf

### MODEL PARAMETERS ###
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32
IMAGE_SIZE = (224, 224)
EPOCHS = 100
INPUT_SHAPE = (224, 224, 3)
METRICS = ['AUC', 'accuracy', 'Recall', 'Precision']


#preprocess.py

### GCP DATA STORAGE ###

BUCKET_NAME = 'alzheimers-project-699'
BUCKET_TRAIN_DATA_PATH = 'data/AlzheimersDataset/train'
STORAGE_LOCATION = 'models/model.joblib'

import numpy as np
import tensorflow as tf

def augment(img, label):
    img = tf.image.random_brightness(
        img, 0.2, seed=123
    )
    
    img = tf.image.stateless_random_contrast(
        img, 0.2, 0.5, seed=(1,2)
    )
        
    return img, label

def preprocessing(img, label):
    img = img / 255
    #img = augment(img)
    return img, label

##trainer.py
import tensorflow as tf
import joblib
import os
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from google.cloud import storage


class Trainer():
    def __init__(self):
        self.model = None
        self.train_path = None
        self.test_path = None
        self.train_generator = None
        self.validation_generator = None
        self.test_generator = None

### MODEL PIPELINE ###
    #def set_data(self):
     #   parent_path = os.path.dirname(os.path.dirname(__file__))
      #  train_path = os.path.join(parent_path, 'raw_data/AlzheimersDataset/train')
       # test_path = os.path.join(parent_path, 'raw_data/AlzheimersDataset/test')
        #self.train_path = train_path
        #self.test_path = test_path

    def load_data(self):
        # Get and preprocess data
        self.train_generator, self.validation_generator = train_data_loading('/content/drive/MyDrive/LeWagon/AlzheimersDataset/train')
        self.test_generator = test_data_loading('/content/drive/MyDrive/LeWagon/AlzheimersDataset/test')

    def set_model(self):
        # Autotune the process
        AUTOTUNE = tf.data.experimental.AUTOTUNE

        self.train_generator = self.train_generator.map(preprocessing, num_parallel_calls=AUTOTUNE)
        self.train_generator = self.train_generator.map(augment, num_parallel_calls=AUTOTUNE)
        self.validation_generator = self.validation_generator.map(preprocessing, num_parallel_calls=AUTOTUNE)
        self.test_generator = self.test_generator.map(preprocessing, num_parallel_calls=AUTOTUNE)

        self.train_generator = self.train_generator.cache().prefetch(buffer_size=AUTOTUNE)
        self.validation_generator = self.validation_generator.cache().prefetch(buffer_size=AUTOTUNE)
        self.test_generator = self.test_generator.cache().prefetch(buffer_size=AUTOTUNE)
        
        ######sel.train_generator

        self.model = build_compile_model()
        return self.model

    def fit_model(self):
        if self.model == None:
            self.set_model()
        es = EarlyStopping(patience=10, restore_best_weights=True)
        rop = ReduceLROnPlateau(monitor='val_loss', factor=0.005, patience=10, min_lr=0.005)
        self.model.fit(self.train_generator, validation_data=self.validation_generator, epochs=50, callbacks=[es, rop], verbose=1)


    def evaluate(self):
        """evaluates the model on test set and returns the Loss, AUC, Accuracy, Recall and Precision"""
        test_scores = self.model.evaluate(self.test_generator)
        scores_dict = {'Loss': test_scores[0],
                       'AUC': test_scores[1],
                       'Accuracy': test_scores[2],
                       'Recall': test_scores[3],
                       'Precision': test_scores[4]}
        return scores_dict


### SAVE MODEL TO GCP ###
    def save_model(self):
        """ Save the trained model into a model.joblib file """
        joblib.dump(self.pipeline, 'model.joblib')
        client = storage.Client()
        bucket = client.bucket(BUCKET_NAME)
        blob = bucket.blob(STORAGE_LOCATION)
        blob.upload_from_filename('model.joblib')
        print(f"uploaded model.joblib to gcp cloud storage under \n => {STORAGE_LOCATION}")


    # Train model and save to gcp




In [ ]:
img = '/content/drive/MyDrive/LeWagon/AlzheimersDataset/train/MildDemented/mildDem0.jpg'

In [ ]:
import PIL

example=PIL.Image.open(img)

exp=np.asarray(example)
exp=exp[np.newaxis,np.newaxis,:]
exp.shape

(1, 1, 208, 176)

In [ ]:
def test(img):
  img=img/2,
  return img

In [ ]:
test(exp).shape

AttributeError: ignored

In [ ]:
t = Trainer()

t.load_data()
t.set_model()
t.fit_model()
t.evaluate()

Found 5121 files belonging to 4 classes.
Using 4097 files for training.
Found 5121 files belonging to 4 classes.
Using 1024 files for validation.
Found 1279 files belonging to 4 classes.
Epoch 1/50
129/129 [==============================] - 363s 3s/step - loss: 2.6635 - auc: 0.7946 - accuracy: 0.5794 - recall: 0.5555 - precision: 0.5927 - val_loss: 1.5891 - val_auc: 0.7908 - val_accuracy: 0.4619 - val_recall: 0.4443 - val_precision: 0.4591
Epoch 2/50
129/129 [==============================] - 37s 284ms/step - loss: 0.8574 - auc: 0.8995 - accuracy: 0.6981 - recall: 0.6717 - precision: 0.7184 - val_loss: 1.6585 - val_auc: 0.7609 - val_accuracy: 0.5928 - val_recall: 0.5674 - val_precision: 0.6434
Epoch 3/50
129/129 [==============================] - 37s 284ms/step - loss: 0.6698 - auc: 0.9338 - accuracy: 0.7693 - recall: 0.7520 - precision: 0.7832 - val_loss: 0.6154 - val_auc: 0.9378 - val_accuracy: 0.7812 - val_recall: 0.7559 - val_precision: 0.8012
Epoch 4/50
129/129 [==================

{'AUC': 0.8717904686927795,
 'Accuracy': 0.6325253844261169,
 'Loss': 1.0146764516830444,
 'Precision': 0.6455591917037964,
 'Recall': 0.613760769367218}

In [ ]:
x,y=next(iter(t.train_generator))

In [ ]:
x.shape

TensorShape([32, 224, 224, 3])

In [ ]:
x[0]

<tf.Tensor: shape=(224, 224, 3), dtype=float32, numpy=
array([[[-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        ...,
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621]],

       [[-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        ...,
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621]],

       [[-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        ...,
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621],
        [-0.02682621, -0.02682621, -0.02682621]],

       ...,

     